In [91]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from skimage import io, color
import requests
from io import BytesIO
from IPython.display import Video

def download_image(url):
    response = requests.get(url)
    img = io.imread(BytesIO(response.content))
    return color.rgb2gray(img)  # Convert to grayscale

def measure_error(U, S, V, original_img, rank):
    S_truncated = np.zeros_like(S)
    S_truncated[:rank] = S[:rank]
    reconstructed_img = np.dot(U, np.dot(np.diag(S_truncated), V))
    relative_error = np.linalg.norm(original_img - reconstructed_img) / np.linalg.norm(original_img)
    return relative_error

def update_plot(i, img_plot, error_plot, U, S, V, original_img, errors, ranks, ax1, ax2, mid_rank, ylim_low, ylim_high):
    # Adjust rank based on the frame index
    if i < mid_rank:
        rank = i + 1
    else:
        rank = mid_rank + (i - mid_rank + 1) * 10

    S_truncated = np.zeros_like(S)
    S_truncated[:rank] = S[:rank]
    reconstructed_img = np.dot(U, np.dot(np.diag(S_truncated), V))

    # Calculate relative error
    relative_error = np.linalg.norm(original_img - reconstructed_img) / np.linalg.norm(original_img)
    errors.append(relative_error)
    ranks.append(rank)

    # Update the image plot and title
    img_plot.set_data(reconstructed_img)
    ax1.set_title(f"Image compression with SVD\n Rank {rank}; Relative Error {relative_error:.2f}")
    # Remove axis ticks and labels from the first subplot (ax1)
    ax1.set_xticks([])
    ax1.set_yticks([])

    # Update the error plot
    error_plot.set_data(ranks, errors)
    ax2.set_xlim(1, len(S))
    ax2.grid(linestyle=":")
    ax2.set_ylim(ylim_low, ylim_high)
    ax2.set_ylabel('Relative Error')
    ax2.set_xlabel('Rank')
    ax2.semilogy()

    # Set xticks to show rank numbers
    ax2.set_xticks(range(1, len(S)+1, max(len(S)//10, 1)))
    plt.tight_layout()

    return img_plot, error_plot

def create_animation(image, filename='svd_animation.mp4'):
    U, S, V = np.linalg.svd(image, full_matrices=False)
    mid_rank = int(0.1 * len(S))  # 10% of the maximum possible rank
    errors = []
    ranks = []

    # Measure errors for minimal and almost maximal rank
    error_min_rank = measure_error(U, S, V, image, 1)
    error_max_rank = measure_error(U, S, V, image, len(S) - 2)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 8))
    img_plot = ax1.imshow(image, cmap='gray', animated=True)
    error_plot, = ax2.plot([], [], 'r-', animated=True)  # Initial empty plot for errors

    # Add watermark
    ax1.text(1, 1.02, '@fminxyz', transform=ax1.transAxes, color='gray', va='bottom', ha='right', fontsize=9)

    # Determine frames for the animation
    initial_frames = list(range(mid_rank))  # First set of frames up to the 10% mark
    subsequent_frames = list(range(mid_rank, len(S), 10))  # Every 10th rank after 10% mark
    frames = initial_frames + subsequent_frames

    ani = animation.FuncAnimation(fig, update_plot, frames=len(frames), fargs=(img_plot, error_plot, U, S, V, image, errors, ranks, ax1, ax2, mid_rank, error_max_rank, error_min_rank), interval=50, blit=True)
    ani.save(filename, writer='ffmpeg', fps=8, dpi=300)
    plt.close(fig)  # Close the figure to prevent it from displaying in the output
    return filename

# URL of the image
# url = "https://pbs.twimg.com/media/GDkqo68WUAAhFxe?format=jpg&name=large"
url = "https://nadezhdin2024.ru/share.jpg"

# Download the image and create the animation
image = download_image(url)
filename = create_animation(image)

# Display the animation inline
Video(filename)

/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_24111/3991610368.py:55: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_24111/3991610368.py:55: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [92]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from skimage import io
import requests
from io import BytesIO
from IPython.display import Video

def download_image(url):
    response = requests.get(url)
    img = io.imread(BytesIO(response.content))
    return img  # Return the original color image

def measure_error(U, S, V, original_img, rank):
    errors = []
    for i in range(3):  # Process each color channel
        S_truncated = np.zeros_like(S[i])
        S_truncated[:rank] = S[i][:rank]
        reconstructed_img = np.dot(U[i], np.dot(np.diag(S_truncated), V[i]))
        error = np.linalg.norm(original_img[:,:,i] - reconstructed_img) / np.linalg.norm(original_img[:,:,i])
        errors.append(error)
    return np.mean(errors)  # Return the average error across channels

def update_plot(i, img_plot, error_plot, U, S, V, original_img, errors, ranks, ax1, ax2, mid_rank, ylim_low, ylim_high):
    reconstructed_img = np.zeros_like(original_img)
    rank = i + 1 if i < mid_rank else mid_rank + (i - mid_rank + 1) * 10

    for j in range(3):  # Process each color channel
        S_truncated = np.zeros_like(S[j])
        S_truncated[:rank] = S[j][:rank]
        reconstructed_img[:,:,j] = np.dot(U[j], np.dot(np.diag(S_truncated), V[j]))

    relative_error = measure_error(U, S, V, original_img, rank)
    errors.append(relative_error)
    ranks.append(rank)

    img_plot.set_data(reconstructed_img)
    ax1.set_title(f"Image compression with SVD\n Rank {rank}; Relative Error {relative_error:.2f}")
    ax1.set_xticks([])
    ax1.set_yticks([])

    error_plot.set_data(ranks, errors)
    ax2.set_xlim(1, len(S[0]))
    ax2.grid(linestyle=":")
    ax2.set_ylim(ylim_low, ylim_high)
    ax2.set_ylabel('Relative Error')
    ax2.set_xlabel('Rank')
    ax2.semilogy()
    ax2.set_xticks(range(1, len(S[0])+1, max(len(S[0])//10, 1)))
    plt.tight_layout()

    return img_plot, error_plot

def create_animation(image, filename='svd_animation_channel_independent.mp4'):
    U, S, V = [], [], []
    for i in range(3):  # Decompose each color channel separately
        ui, si, vi = np.linalg.svd(image[:,:,i], full_matrices=False)
        U.append(ui)
        S.append(si)
        V.append(vi)

    mid_rank = int(0.1 * len(S[0]))
    errors, ranks = [], []

    error_min_rank = measure_error(U, S, V, image, 1)
    error_max_rank = measure_error(U, S, V, image, len(S[0]) - 2)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 8))
    img_plot = ax1.imshow(image, animated=True)
    error_plot, = ax2.plot([], [], 'r-', animated=True)

    ax1.text(1, 1.02, '@fminxyz', transform=ax1.transAxes, color='gray', va='bottom', ha='right', fontsize=9)

    initial_frames = list(range(mid_rank))
    subsequent_frames = list(range(mid_rank, len(S[0]), 10))
    frames = initial_frames + subsequent_frames

    ani = animation.FuncAnimation(fig, update_plot, frames=len(frames), fargs=(img_plot, error_plot, U, S, V, image, errors, ranks, ax1, ax2, mid_rank, error_max_rank, error_min_rank), interval=50, blit=True)
    ani.save(filename, writer='ffmpeg', fps=8, dpi=300)
    plt.close(fig)
    return filename

# URL of the image
url = "https://nadezhdin2024.ru/share.jpg"

# Download the image and create the animation
image = download_image(url)
filename = create_animation(image)

# Display the animation inline
Video(filename)


/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_24111/1483067763.py:50: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_24111/1483067763.py:50: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [90]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from skimage import io, color
import requests
from io import BytesIO
from IPython.display import Video

def download_image(url):
    response = requests.get(url)
    img = io.imread(BytesIO(response.content))
    shape = img.shape
    return np.hstack([img[:, :, i] for i in range(img.shape[2])]), shape

def measure_error(U, S, V, original_img, rank):
    S_truncated = np.zeros_like(S)
    S_truncated[:rank] = S[:rank]
    reconstructed_img = np.dot(U, np.dot(np.diag(S_truncated), V))
    relative_error = np.linalg.norm(original_img - reconstructed_img) / np.linalg.norm(original_img)
    return relative_error

def update_plot(i, img_plot, error_plot, U, S, V, original_img, errors, ranks, ax1, ax2, mid_rank, ylim_low, ylim_high):
    # Adjust rank based on the frame index
    if i < mid_rank:
        rank = i + 1
    else:
        rank = mid_rank + (i - mid_rank + 1) * 10

    S_truncated = np.zeros_like(S)
    S_truncated[:rank] = S[:rank]
    reconstructed_img = np.dot(U, np.dot(np.diag(S_truncated), V))

    # Calculate relative error
    relative_error = np.linalg.norm(original_img - reconstructed_img) / np.linalg.norm(original_img)
    errors.append(relative_error)
    ranks.append(rank)

    channels = [reconstructed_img[:, i*shape[1]:(i+1)*shape[1]] for i in range(shape[2])]
    img_c = np.dstack(channels)
    img_c = (img_c-np.min(img_c))/(np.max(img_c)-np.min(img_c))

    # Update the image plot and title
    img_plot.set_data(img_c)
    ax1.set_title(f"Image compression with SVD\n Rank {rank}; Relative Error {relative_error:.2f}")
    # Remove axis ticks and labels from the first subplot (ax1)
    ax1.set_xticks([])
    ax1.set_yticks([])

    # Update the error plot
    error_plot.set_data(ranks, errors)
    ax2.set_xlim(1, len(S))
    ax2.grid(linestyle=":")
    ax2.set_ylim(ylim_low, ylim_high)
    ax2.set_ylabel('Relative Error')
    ax2.set_xlabel('Rank')
    ax2.semilogy()

    # Set xticks to show rank numbers
    ax2.set_xticks(range(1, len(S)+1, max(len(S)//10, 1)))
    plt.tight_layout()

    return img_plot, error_plot

def create_animation(image, shape, filename='svd_animation_stack.mp4'):
    U, S, V = np.linalg.svd(image, full_matrices=False)
    mid_rank = int(0.1 * len(S))  # 10% of the maximum possible rank
    errors = []
    ranks = []

    # Measure errors for minimal and almost maximal rank
    error_min_rank = measure_error(U, S, V, image, 1)
    error_max_rank = measure_error(U, S, V, image, len(S) - 2)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 8))

    channels = [image[:, i*shape[1]:(i+1)*shape[1]] for i in range(shape[2])]
    img_c = np.dstack(channels)
    img_c = img_c/np.max(img_c)

    img_plot = ax1.imshow(img_c, cmap='gray', animated=True)
    error_plot, = ax2.plot([], [], 'r-', animated=True)  # Initial empty plot for errors

    # Add watermark
    ax1.text(1, 1.02, '@fminxyz', transform=ax1.transAxes, color='gray', va='bottom', ha='right', fontsize=9)

    # Determine frames for the animation
    initial_frames = list(range(mid_rank))  # First set of frames up to the 10% mark
    subsequent_frames = list(range(mid_rank, len(S), 10))  # Every 10th rank after 10% mark
    frames = initial_frames + subsequent_frames

    ani = animation.FuncAnimation(fig, update_plot, frames=len(frames), fargs=(img_plot, error_plot, U, S, V, image, errors, ranks, ax1, ax2, mid_rank, error_max_rank, error_min_rank), interval=50, blit=True)
    ani.save(filename, writer='ffmpeg', fps=8, dpi=300)
    plt.close(fig)  # Close the figure to prevent it from displaying in the output
    return filename

# URL of the image
# url = "https://pbs.twimg.com/media/GDkqo68WUAAhFxe?format=jpg&name=large"
url = "https://nadezhdin2024.ru/share.jpg"

# Download the image and create the animation
image, shape = download_image(url)
filename = create_animation(image, shape)

# Display the animation inline
Video(filename)

/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_24111/936064363.py:60: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/7m/3rbdnx5n5sz625f3l87m91cc0000gn/T/ipykernel_24111/936064363.py:60: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
